In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

In [ ]:
def generate_calibration_diagrams(screen_width=1920, screen_height=1080, num_points=50, error_margin=150):
    # Generate random gaze points
    x = np.random.randint(0, screen_width, num_points)
    y = np.random.randint(0, screen_height, num_points)
    points = np.column_stack((x, y))

    # Calculate convex hull
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]

    # Function to plot screen boundaries
    def plot_screen_boundaries():
        plt.plot([0, screen_width, screen_width, 0, 0], 
                 [0, 0, screen_height, screen_height, 0], 'k--')

    # 1. Convex Hull of Captured Gaze Points
    plt.figure(figsize=(10, 6))
    plt.scatter(x, y, c='blue', label='Gaze Points')
    for simplex in hull.simplices:
        plt.plot(points[simplex, 0], points[simplex, 1], 'r-')
    plot_screen_boundaries()
    plt.title('Convex Hull of Captured Gaze Points')
    plt.xlabel('Screen Width (pixels)')
    plt.ylabel('Screen Height (pixels)')
    plt.legend()
    plt.gca().set_aspect('equal', adjustable='box')
    plt.savefig('convex_hull.png')
    plt.close()

    # 2. Error Margin Extension
    plt.figure(figsize=(10, 6))
    plt.scatter(x, y, c='blue', label='Gaze Points')
    for simplex in hull.simplices:
        plt.plot(points[simplex, 0], points[simplex, 1], 'r-')
    
    # Calculate and plot extended hull
    center = np.mean(hull_points, axis=0)
    extended_hull_points = hull_points + error_margin * (hull_points - center) / np.linalg.norm(hull_points - center, axis=1)[:, np.newaxis]
    extended_hull_points = np.vstack((extended_hull_points, extended_hull_points[0]))  # Close the polygon
    plt.plot(extended_hull_points[:, 0], extended_hull_points[:, 1], 'g--', label='Error Margin')
    
    plot_screen_boundaries()
    plt.title('Error Margin Extension')
    plt.xlabel('Screen Width (pixels)')
    plt.ylabel('Screen Height (pixels)')
    plt.legend()
    plt.gca().set_aspect('equal', adjustable='box')
    plt.savefig('error_margin_extension.png')
    plt.close()

    # 3. Final Calibration Points
    plt.figure(figsize=(10, 6))
    plot_screen_boundaries()
    
    # Clip extended hull to screen boundaries
    clipped_hull = np.clip(extended_hull_points, [0, 0], [screen_width, screen_height])
    plt.plot(clipped_hull[:, 0], clipped_hull[:, 1], 'g-', label='Clipped Extended Hull')
    
    # Find corners of clipped hull
    min_x, min_y = np.min(clipped_hull, axis=0)
    max_x, max_y = np.max(clipped_hull, axis=0)
    corners = np.array([[min_x, min_y], [max_x, min_y], [max_x, max_y], [min_x, max_y]])
    
    # Plot corners
    plt.scatter(corners[:, 0], corners[:, 1], c='red', s=100, marker='x', label='Calibration Points')
    
    for i, (x, y) in enumerate(corners):
        plt.annotate(f'Point {i+1}', (x, y), xytext=(5, 5), textcoords='offset points')
    
    plt.title('Final Calibration Points')
    plt.xlabel('Screen Width (pixels)')
    plt.ylabel('Screen Height (pixels)')
    plt.legend()
    plt.gca().set_aspect('equal', adjustable='box')
    plt.savefig('final_calibration_points.png')
    plt.close()

# Generate the diagrams
generate_calibration_diagrams()